In [ ]:
# from joblib import Parallel, delayed
# import numpy as np
# import pandas as pd
# from sklearn.metrics.pairwise import cosine_similarity

# # ===============================================================
# # Updated per-domain matching with embeddings + cluster features
# # ===============================================================

# def process_one_domain(domain, resume_df, job_posts_df, resume_embeddings, job_embeddings):
#     domain_resumes = resume_df[resume_df["ClusterDomainLabel"] == domain]
#     domain_jobs = job_posts_df[job_posts_df["ClusterDomainLabel"] == domain]
#     results = []

#     if domain_jobs.empty or domain_resumes.empty:
#         return results

#     print(f"Processing domain: {domain} ({len(domain_jobs)} jobs × {len(domain_resumes)} resumes)")

#     for j_idx, job_row in domain_jobs.iterrows():
#         job_text = job_row["job_text_clean"]
#         j_emb = job_embeddings[j_idx]
#         job_cluster = job_row["PredictedCluster"]
#         job_domain = job_row["ClusterDomainLabel"]

#         for r_idx, resume_row in domain_resumes.iterrows():
#             resume_text = resume_row["Resume_clean"]
#             r_emb = resume_embeddings[r_idx]
#             resume_cluster = resume_row["PredictedCluster"]
#             resume_domain = resume_row["ClusterDomainLabel"]

#             # --- Semantic cosine similarity ---
#             cos_sim = cosine_similarity([r_emb], [j_emb])[0][0]

#             # --- Hybrid composite scoring ---
#             final_score, components = calculate_composite_score(
#                 resume_text, job_text,
#                 r_emb, j_emb,
#                 resume_cluster, job_cluster,
#                 resume_domain, job_domain
#             )

#             results.append({
#                 "resume_id": resume_row.get("ID", r_idx),
#                 "job_id": job_row.get("ID", j_idx),
#                 "resume_cluster": resume_cluster,
#                 "job_cluster": job_cluster,
#                 "same_cluster": int(resume_cluster == job_cluster),
#                 "resume_domain": resume_domain,
#                 "job_domain": job_domain,
#                 "same_domain": 1,  # all are domain-aligned here
#                 "cosine_similarity": cos_sim,
#                 **components,
#                 "final_score": final_score
#             })
#     return results


# # ===============================================================
# # Parallel wrapper for all domains
# # ===============================================================
# def generate_training_pairs_parallel(resume_df, job_posts_df, resume_embeddings, job_embeddings):
#     domains = sorted(set(resume_df["ClusterDomainLabel"]) & set(job_posts_df["ClusterDomainLabel"]))
    
#     all_results = Parallel(n_jobs=-1, backend="loky")(
#         delayed(process_one_domain)(domain, resume_df, job_posts_df, resume_embeddings, job_embeddings)
#         for domain in domains
#     )

#     pairs_flat = [x for sublist in all_results for x in sublist]
#     training_pairs = pd.DataFrame(pairs_flat)
#     print(f"\n✅ Generated {len(training_pairs):,} resume–job pairs across {len(domains)} domains.")
#     return training_pairs


# # ===============================================================
# # Run the generator
# # ===============================================================
# training_pairs = generate_training_pairs_parallel(
#     resumes_filtered, jobs_filtered, resume_embeddings, job_embeddings
# )

# # training_pairs = generate_training_pairs_parallel(resume_df, job_posts_df, resume_embeddings, job_embeddings)

# # ===============================================================
# # Labeling: 3 classes (good / medium / poor)
# # ===============================================================
# scores = training_pairs["final_score"]
# high_t = scores.quantile(0.75)
# low_t = scores.quantile(0.25)

# def assign_label(score):
#     if score >= high_t:
#         return 1          # good fit
#     elif score <= low_t:
#         return 0          # poor fit
#     else:
#         return 0.5        # medium fit

# training_pairs["label"] = scores.apply(assign_label)
# print(f"Label thresholds: High ≥ {high_t:.3f}, Low ≤ {low_t:.3f}")

# # ===============================================================
# # Save and visualize
# # ===============================================================
# out_path = os.path.join(DATA_DIR, "resume_job_training_data.csv")
# training_pairs.to_csv(out_path, index=False)
# print(f"✅ Saved labeled training data → {out_path}")

# # import matplotlib.pyplot as plt
# # plt.figure(figsize=(8,5))
# # plt.hist(training_pairs["final_score"], bins=30, color='skyblue', edgecolor='black')
# # plt.title("Distribution of Resume–Job Final Fit Scores")
# # plt.xlabel("Final Score")
# # plt.ylabel("Frequency")
# # plt.show()
